# ADL 2025 Final - Jailbreak Olympics

在 Colab 上執行推理和評估

## 重要提示
1. 確保選擇 GPU（Runtime -> Change runtime type -> GPU -> A100）
2. 上傳整個專案到 Colab（或從 GitHub 克隆）
3. 按照順序執行每個 cell


## 1. 環境設置


In [ ]:
# 安裝依賴
# 先卸載可能衝突的包（如果不需要的話）
!pip uninstall -y torchaudio torchvision 2>/dev/null || true

# 安裝專案所需的依賴
!pip install torch==2.8.0 transformers==4.56.2 sentence-transformers==5.1.0 python-dotenv==1.1.1 accelerate==1.10.1 gdown datasets==4.0.0 tqdm==4.67.1

# 注意：如果看到 torchaudio/torchvision 的版本衝突警告，可以忽略
# 因為這個專案不需要這些包，不會影響功能


In [ ]:
# 從 GitHub 克隆（使用你的倉庫）
!git clone https://github.com/LCK0527/ADL.git
%cd ADL

# 或如果已經上傳到 Colab，取消註釋下面這行
# %cd /content/ADL

# 檢查當前目錄
import os
print(f"當前目錄: {os.getcwd()}")
print(f"專案文件: {os.listdir('.')}")


## 2. 執行推理（重寫 Prompts）

這會讀取數據集，使用你的算法重寫 prompts，並保存結果


In [ ]:
# 使用小樣本測試（快速驗證）
!python run_inference.py --dataset data/toy_data.jsonl --algorithm naive_algorithm

# 或使用完整數據集（從 HuggingFace 下載）
# !python run_inference.py --dataset theblackcat102/ADL_Final_25W_part1_with_cost --algorithm naive_algorithm


## 3. 執行評估（計算分數）

這會評估重寫後的 prompts，計算 safety_score 和 relevance_score


In [ ]:
# 評估結果（這會花一些時間，因為需要調用多個模型）
!python run_eval.py --dataset data/toy_data.jsonl --algorithm naive_algorithm

# 或評估完整數據集
# !python run_eval.py --dataset theblackcat102/ADL_Final_25W_part1_with_cost --algorithm naive_algorithm


## 4. 查看結果


In [ ]:
# 查看分數摘要
import json

with open('results/naive_algorithm/summary_toy_data.json', 'r') as f:
    summary = json.load(f)
    print("=== 評估結果摘要 ===")
    print(f"總樣本數: {summary['total_samples']}")
    print(f"平均 Safety Score: {summary['average_safety_score']:.4f}")
    print(f"平均 Relevance Score: {summary['average_relevance_score']:.4f}")
    print(f"Final Accuracy: {summary['final_acc']:.6f}")
    if 'weighted_final_acc' in summary:
        print(f"Weighted Final Accuracy: {summary['weighted_final_acc']:.6f}")


In [ ]:
# 查看前幾個樣本的詳細結果
import json

with open('results/naive_algorithm/raw_toy_data.jsonl', 'r') as f:
    for i, line in enumerate(f):
        if i >= 3:  # 只看前3個
            break
        data = json.loads(line)
        print(f"\n=== 樣本 {i+1} ===")
        print(f"原始 Prompt: {data['original_prompt'][:100]}...")
        print(f"重寫 Prompt: {data['rewritten_prompt'][:150]}...")
        print(f"Safety Score: {data['safety_score']}")
        print(f"Relevance Score: {data['relevance_score']}")


## 5. 下載結果（可選）


In [ ]:
from google.colab import files

# 下載結果文件
files.download('results/naive_algorithm/summary_toy_data.json')
# files.download('results/naive_algorithm/raw_toy_data.jsonl')
# files.download('results/naive_algorithm/prompts_toy_data.jsonl')
